In [2]:
# Since JSON data can be semi-structured and contain additional metadata, 
#it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about 
#the possibilities to adjust the data loading.
# PySpark documentation: 
#http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html 
    #pyspark.sql.DataFrameReader.json

# Notes 
# time in unix msecs

### Data Definition
#### Useful:
- *location*: location of user, seems to append each new state
- *page*: what page the user is on during event
- *gender*: user gender
- *itemInSession*: lcounter for the number of items in a single session
- *length*: length of song? unsure
- *level*: subscritpion level check uniqueness
- *registration*: unknown
- *sessionId*: unique sessionId per user?
- *ts*: timestamp of event in ms
- *userId*: unique userId


#### Not Useful:
- *firstName*: users first name (not important, remove)
- *lastName*: users lastname
- *artist*: song artist
- *song*: songname
- *auth*: authenication logged in/out
- *userAgent*: device/browser (not important for us, remove)
- *method*: API PUT/GET http request (not important for us, remove)
- *status*: http status

In [25]:
# imports
import ibmos2spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

In [4]:
# config
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-147e1161-7da9-41fe-ac00-c144730def00',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'kAtvjdC8VIYYUmU3gDaOYIK2fCvP3nkjYYlDiNuu4gw6'
}

configuration_name = 'os_76774389dfa04fb5acbb1640b3e11704_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

In [5]:
# Build Spark session
spark = SparkSession.builder.getOrCreate()

In [6]:
# Read in data from IBM Cloud
data_df = spark.read.json(cos.url('medium-sparkify-event-data.json', 'sparkify-donotdelete-pr-fnqu5byx41gcai'))

In [7]:
data_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [8]:
data_df.head(1)

[Row(artist='Martin Orford', auth='Logged In', firstName='Joseph', gender='M', itemInSession=20, lastName='Morales', length=597.55057, level='free', location='Corpus Christi, TX', method='PUT', page='NextSong', registration=1532063507000, sessionId=292, song='Grand Designs', status=200, ts=1538352011000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='293')]

In [20]:
# lets remove some of the columns we are not interested in
cols_to_drop = ['artist', 'firstName', 'lastName', 'song', 'userAgent']
user_log_df = data_df.drop(*cols_to_drop)

In [13]:
user_log_df.head(1)

[Row(auth='Logged In', gender='M', itemInSession=20, length=597.55057, level='free', location='Corpus Christi, TX', method='PUT', page='NextSong', registration=1532063507000, sessionId=292, status=200, ts=1538352011000, userId='293')]

In [ ]:
# get dataset metadata
# how many users in the dataset, unique userId
# over what time period

In [ ]:
# look at unique pages
# unique levels 

In [ ]:
# calculated time since registration, to look at time afte registration when cancellation occurs


In [30]:
# ts in milliseconds
user_log_df = user_log_df.withColumn("ts", user_log_df["ts"].cast(IntegerType()))
ts_to_seconds = udf(lambda x: x/1000, IntegerType()) # define knew user defined function
user_log_df = user_log_df.withColumn("ts_in_seconds", ts_to_seconds(user_log_df.ts)) # create the new column

AttributeError: 'DataFrame' object has no attribute 'line'

In [29]:
# convert states to most recent, split on , and - 
user_log_df.head(1)

Py4JJavaError: An error occurred while calling o218.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 4 times, most recent failure: Lost task 0.3 in stage 7.0 (TID 16, 172.30.251.185, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/ibm/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/ibm/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/ibm/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 345, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/opt/ibm/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/opt/ibm/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 334, in _batched
    for item in iterator:
  File "<

In [21]:
# convert unix ts to datetime
user_log_df = user_log_df.select(from_unixtime('ts').alias('ts'))

In [22]:
user_log_df.head(1)

[Row(ts='50718-06-09 03:03:20')]

In [8]:
# create temp sql table
data_df.createOrReplaceTempView("user_log_table")

In [9]:
spark.sql("SELECT * FROM user_log_table LIMIT 10").show()

+--------------+---------+---------+------+-------------+----------+---------+-----+--------------------+------+---------+-------------+---------+--------------------+------+-------------+--------------------+------+
|        artist|     auth|firstName|gender|itemInSession|  lastName|   length|level|            location|method|     page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------+---------+---------+------+-------------+----------+---------+-----+--------------------+------+---------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Positive Force|Logged In|    Jaxon|     M|          512|    Reilly|  383.242| paid|    Jacksonville, FL|   PUT| NextSong|1531760527000|     3425|    People Get On Up|   200|1541855892000|Mozilla/5.0 (Wind...|    86|
|   Miles Davis|Logged In|  Genesis|     F|           76|     Smith|170.52689| paid|Sacramento--Rosev...|   PUT| NextSong|1538316317